In [1]:
import chess
import chess.engine
import numpy as np
import tqdm
import pandas as pd
from stockfish import Stockfish
import tensorflow as tf
import keras
import chess.pgn
import random

In [17]:
'''
	c >= 5, white is winning  ==0
	5 > c >= 3, white has a decisive advantage ==1
	3 > c >= 1, white is better  == 2
	1 > c > -1, the position is equal  ==3
	-1 >= c > -3, black is better  ==4
	-3 > c >= -5, black has a decisive advantage  ==5
	-5 >= c, black is winning  ==6 
'''
def label(score):
    if score >= 5:
        return 0
    elif 5 > score >= 3:
        return 1
    elif 3 > score >= 1:
        return 2
    elif 1 > score > -1:
        return 3
    elif -1 >= score > -3:
        return 4
    elif -3 > score >= -5:
        return 5
    elif -5 >= score:
        return 6

In [83]:
pgn=open(r"C:\Users\congcong\Desktop\FYP\strong-players\Ljubojevic Ljubomir (YUG) 2070 Games.PGN")
game = chess.pgn.read_game(pgn)
fen=[]
scores=[]
label_score=[]
engine = chess.engine.SimpleEngine.popen_uci(r"C:\Users\congcong\Desktop\FYP\stockfish_14_win_x64\stockfish_14_x64.exe")

while True:
    game = chess.pgn.read_game(pgn)
    if game is None:
        break
    board = game.board()
    
    for move in game.mainline_moves():
        board.push(move)
        fen.append(board.copy().fen())
        info = engine.analyse(board, chess.engine.Limit(depth=10))
        
        score = info['score'].white().score(mate_score=1000)
        score=score/100.0
        scores.append(score)
        label_score.append(label(score))
score_df=pd.DataFrame(scores,columns=['scores'])
label_s=pd.DataFrame(label_score,columns=['label_score'])
fen_df=pd.DataFrame(fen,columns=['fen'])
csv_file=pd.concat([fen_df,score_df, label_s], axis=1)
csv_file.to_csv('evl_data.csv', encoding='utf-8', header='true',index = False)
print('done')

done


In [84]:

df = pd.read_csv('evl_data.csv')

In [85]:
df.head()

,fen,scores,label_score
0,rnbqkbnr/pppppppp/8/8/5P2/8/PPPPP1PP/RNBQKBNR ...,-0.42,3.0
1,rnbqkbnr/pppp1ppp/8/4p3/5P2/8/PPPPP1PP/RNBQKBN...,0.84,3.0
2,rnbqkbnr/pppp1ppp/8/4P3/8/8/PPPPP1PP/RNBQKBNR ...,0.22,3.0
3,rnbqkbnr/ppp2ppp/3p4/4P3/8/8/PPPPP1PP/RNBQKBNR...,0.54,3.0
4,rnbqkbnr/ppp2ppp/3P4/8/8/8/PPPPP1PP/RNBQKBNR b...,0.32,3.0


In [86]:
df2=df["fen"]
df2

0         rnbqkbnr/pppppppp/8/8/5P2/8/PPPPP1PP/RNBQKBNR ...
1         rnbqkbnr/pppp1ppp/8/4p3/5P2/8/PPPPP1PP/RNBQKBN...
2         rnbqkbnr/pppp1ppp/8/4P3/8/8/PPPPP1PP/RNBQKBNR ...
3         rnbqkbnr/ppp2ppp/3p4/4P3/8/8/PPPPP1PP/RNBQKBNR...
4         rnbqkbnr/ppp2ppp/3P4/8/8/8/PPPPP1PP/RNBQKBNR b...
                                ...                        
161518      8/8/1b6/4kppp/p1p5/B1P2P1P/1P2K1P1/8 w - - 4 46
161519     8/4B3/1b6/4kppp/p1p5/2P2P1P/1P2K1P1/8 b - - 5 46
161520    8/4B3/1b6/4kp1p/p1p3p1/2P2P1P/1P2K1P1/8 w - - ...
161521     8/8/1b6/4kpBp/p1p3p1/2P2P1P/1P2K1P1/8 b - - 1 47
161522       8/8/1b6/4kpBp/p1p5/2P2P1p/1P2K1P1/8 w - - 0 48
Name: fen, Length: 161523, dtype: object

In [87]:
df3=df["scores"]
df3

0        -0.42
1         0.84
2         0.22
3         0.54
4         0.32
          ... 
161518   -0.33
161519   -0.37
161520   -0.36
161521   -0.37
161522   -0.58
Name: scores, Length: 161523, dtype: float64